### 1. You're given two 32-bit numbers M and N, and two bit positions j and i. Insert M into N such that the bits j through i in N will be M. M is guaranteed to fit in bits j through i.
- Sets bits i through j of N as 0 ( Use a mask that is all 1, with bits j through i 0, AND with N)
- Shift M to left by i, so it's aligned with M
- bitwise OR to set j-i bits of N as M

In [1]:
def insert_bits(N, M, i, j):
    # Step 1: Create a mask to clear bits j through i in N
    all_ones = ~0  # This will be a sequence of all 1's
    
    # 1's before position j, then 0's
    left = all_ones << (j + 1)
    
    # 1's after position i
    right = (1 << i) - 1
    
    # All 1's, except for 0's between i and j
    mask = left | right
    
    # Clear bits j through i in N
    N_cleared = N & mask
    
    # Step 2: Shift M to the correct position
    M_shifted = M << i
    
    # Step 3: Combine N_cleared with M_shifted
    result = N_cleared | M_shifted
    
    return result

# Example usage
N = 0b10000000000
M = 0b10011
i = 2
j = 6

result = insert_bits(N, M, i, j)
print(f"Result: {bin(result)}")


Result: 0b10001001100


### 2. Given a real number between 0 and 1 as a double, convert to binary. If unable, show ERROR.
- repeatedly multiply the number by 2 and record the integer part of the result (either 0 or 1). 
- If the integer part is 1, we subtract 1 from the number and continue.

In [2]:
def double_to_binary(num):
    if num >= 1 or num <= 0:
        return "ERROR"

    binary = "."
    frac = num

    while frac > 0:
        # Setting a limit on length: 32 characters
        if len(binary) >= 34:  # 1 for the decimal point + 32 for binary places + 1 initial 0
            return "ERROR"

        r = frac * 2
        if r >= 1:
            binary += "1"
            frac = r - 1
        else:
            binary += "0"
            frac = r

    return binary

# Example usage
number = 0.625
result = double_to_binary(number)
print(f"Binary representation of {number}: {result}")

number = 0.1
result = double_to_binary(number)
print(f"Binary representation of {number}: {result}")


Binary representation of 0.625: .101
Binary representation of 0.1: ERROR


### 3. Given an integer, find the length of the longest sequence of 1s you can create by flipping 1 bit.
- Traverse through the bits of the integer.
- Keep track of the lengths of sequences of 1s before and after a possible 0 bit that can be flipped.
- Calculate the maximum possible length of 1s that can be obtained by flipping each 0 bit to 1.


In [3]:
def longest_sequence_of_1s(num):
    # If all bits are 1, then flipping any bit won't change the length of 1s
    if ~num == 0:
        return num.bit_length()
    
    current_length = 0
    previous_length = 0
    max_length = 0
    
    while num != 0:
        if (num & 1) == 1:  # Current bit is 1
            current_length += 1
        else:  # Current bit is 0
            # If next bit is also 0, reset the previous length to 0
            previous_length = 0 if (num & 2) == 0 else current_length
            current_length = 0
        
        # max_length is the maximum of current found lengths
        max_length = max(max_length, previous_length + current_length + 1)
        
        # Right shift num by 1 to check the next bit
        num >>= 1
    
    return max_length

# Example usage
number = 1775  # Binary: 11011101111
print(f"The length of the longest sequence of 1s by flipping one bit is: {longest_sequence_of_1s(number)}")


The length of the longest sequence of 1s by flipping one bit is: 8


### 4. Given a positive number, print the next smallest and largest numbers with the same number of 1s in their binary representation.
- To find the next largest number with the same number of 1s:

. Identify the rightmost non-trailing zero and the rightmost 1 that follows it.
. Swap these bits.
. Rearrange all bits to the right of the swap position to be in ascending order (to get the smallest possible number with the same number of 1s).

- To find the next smallest number with the same number of 1s:

. Identify the rightmost non-trailing one and the rightmost 0 that follows it.
. Swap these bits.
. Rearrange all bits to the right of the swap position to be in descending order (to get the largest possible number with the same number of 1s).


In [5]:
def get_next_largest(n):
    c = n
    c0 = c1 = 0
    while ((c & 1) == 0) and (c != 0):
        c0 += 1
        c >>= 1
    while (c & 1) == 1:
        c1 += 1
        c >>= 1
    
    if c0 + c1 == 31 or c0 + c1 == 0:
        return -1  # There is no larger number with the same number of 1s
    
    pos = c0 + c1
    n |= (1 << pos)  # Flip the rightmost non-trailing zero
    n &= ~((1 << pos) - 1)  # Clear all bits to the right of pos
    n |= (1 << (c1 - 1)) - 1  # Insert (c1-1) ones on the right
    
    return n

def get_next_smallest(n):
    temp = n
    c0 = c1 = 0
    while (temp & 1) == 1:
        c1 += 1
        temp >>= 1
    
    if temp == 0:
        return -1  # There is no smaller number with the same number of 1s
    
    while ((temp & 1) == 0) and (temp != 0):
        c0 += 1
        temp >>= 1
    
    p = c0 + c1
    n &= ((~0) << (p + 1))  # Clears from bit p onwards
    mask = (1 << (c1 + 1)) - 1  # Sequence of (c1+1) ones
    n |= mask << (c0 - 1)
    
    return n

# Example usage
number = 13948  # Binary: 11011001111100

next_largest = get_next_largest(number)
next_smallest = get_next_smallest(number)

print(f"Original number: {number} (binary: {bin(number)})")
print(f"Next largest: {next_largest} (binary: {bin(next_largest)})")
print(f"Next smallest: {next_smallest} (binary: {bin(next_smallest)})")


Original number: 13948 (binary: 0b11011001111100)
Next largest: 13967 (binary: 0b11011010001111)
Next smallest: 13946 (binary: 0b11011001111010)


### 5. What does n & (n-1) == 0 mean?

- if a & b == 0, a and b has no bit positions which is both 1. 
- n-1 is same as flipping the trailing zeros to 1 and the last 1 to 0. eg: n = 1000, n-1 = 0111
- Numbers that are powers of 2 have a single 1 bit followed by 0 bits in their binary representation. eg: 1000
- When we perform a bitwise AND operation (&) between n and n-1, the result will be 0 if and only if n is a power of 2. This is because:

    - The 1 bit in n and the 0 bit in n-1 will always be in the same position.
    - All other bits to the right will be 0 in n and 1 in n-1.

### 6. Find the number of bits you'd need to flip to convert integer A to integer B
- by XORing the two numbers, you can identify the bits that differ between them. 
- Then, you simply count the number of 1s in the result to get the number of differing bits.

In [8]:
def count_bits_to_flip(a, b):
    # Perform XOR between a and b
    xor_result = a ^ b
    
    # Count the number of 1s in the XOR result
    count = 0
    while xor_result:
        count += xor_result & 1 # Increment the count if the least significant bit of xor_result is 1.
        xor_result >>= 1 # right shift one bit to check the next bit
    
    return count

# Example usage
A = 29  # Binary: 11101
B = 15  # Binary: 01111

print(f"Number of bits to flip to convert {A} to {B}: {count_bits_to_flip(A, B)}")


Number of bits to flip to convert 29 to 15: 2


### 7. Write a function to swap odd and even bits in as few steps are possible.

In [9]:
def swap_odd_even_bits(n):
    # Masks for odd and even bits
    ODD_MASK = 0xAAAAAAAA  # Binary: 10101010... (32 bits)
    EVEN_MASK = 0x55555555  # Binary: 01010101... (32 bits)
    
    # Extract odd bits and shift them right
    odd_bits = (n & ODD_MASK) >> 1
    
    # Extract even bits and shift them left
    even_bits = (n & EVEN_MASK) << 1
    
    # Combine the shifted odd and even bits
    return odd_bits | even_bits

# Example usage
number = 23  # Binary: 10111
swapped = swap_odd_even_bits(number)
print(f"Original number: {number} (binary: {bin(number)})")
print(f"Swapped number: {swapped} (binary: {bin(swapped)})")


Original number: 23 (binary: 0b10111)
Swapped number: 43 (binary: 0b101011)


### 8. A monochrome screen is stored as a single array of bytes, allowing eight consecutive pixels to be stored in one byte.The screen has width w,where w is divisible by 8 (that is,no byte will be split across rows). The height of the screen,of course, can be derived from the length of the array and the width. 
- Implement a function drawHorizontall_ine(byte[] screen, int width, int x1, int x2, int y )which draws a horizontal line from (xl, y) to (x2, y).

#### solution:

1. Understanding the Screen Representation:

Each byte represents 8 consecutive pixels.
The screen width w is divisible by 8, meaning each row consists of w / 8 bytes.

2. Calculating the Byte Indices:

To draw a line from (x1, y) to (x2, y), calculate which bytes these pixels fall into.
If x1 and x2 fall into the same byte, we only need to manipulate that byte. Otherwise, we manipulate the bits from x1 to the end of its byte and from the beginning of x2's byte to x2.

3. Bit Manipulation:

Use bitwise operations to set the appropriate bits within the bytes.

In [10]:
def drawHorizontalLine(screen, width, x1, x2, y):
    # Number of bytes in a row
    bytes_per_row = width // 8
    
    # Calculate the start and end byte index in the array
    start_byte = (y * bytes_per_row) + (x1 // 8)
    end_byte = (y * bytes_per_row) + (x2 // 8)
    
    # Set bits for the first byte
    start_offset = x1 % 8
    end_offset = x2 % 8
    
    if start_byte == end_byte:
        # x1 and x2 are within the same byte
        screen[start_byte] |= ((0xFF >> start_offset) & (0xFF << (7 - end_offset)))
    else:
        # Set bits for the start byte
        screen[start_byte] |= 0xFF >> start_offset
        
        # Set bits for the bytes in between
        for i in range(start_byte + 1, end_byte):
            screen[i] |= 0xFF
        
        # Set bits for the end byte
        screen[end_byte] |= 0xFF << (7 - end_offset)

# Example usage
screen = [0b00000000] * 8  # 8 bytes (64 bits) for simplicity, 8x8 screen
width = 8  # Width is 8 pixels
x1 = 1
x2 = 6
y = 2

drawHorizontalLine(screen, width, x1, x2, y)

# Print the screen in binary format for visualization
for row in range(len(screen) // (width // 8)):
    row_data = screen[row * (width // 8):(row + 1) * (width // 8)]
    print(' '.join(format(byte, '08b') for byte in row_data))


00000000
00000000
01111110
00000000
00000000
00000000
00000000
00000000
